#Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import classification_report,f1_score, confusion_matrix, accuracy_score

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/swuds/t5_thesis/98_progress/dataset/nd_sales_LogisticRegression_base.csv')
df.head()

#Preprocess

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')

In [ ]:
df['lag_1'] = df['sales_sum'].shift(1)
df['lag_2'] = df['sales_sum'].shift(2)
df['lag_3'] = df['sales_sum'].shift(3)
df['lag_4'] = df['sales_sum'].shift(4)
df['lag_5'] = df['sales_sum'].shift(5)
df['lag_6'] = df['sales_sum'].shift(6)
df['lag_7'] = df['sales_sum'].shift(7)
df['lag_8'] = df['sales_sum'].shift(8)
df['lag_9'] = df['sales_sum'].shift(9)
df['lag_10'] = df['sales_sum'].shift(10)
df['lag_11'] = df['sales_sum'].shift(11)

In [ ]:
df.dropna(inplace=True)
df.head()

In [ ]:
df['is_Sun'] = (df.index.dayofweek == 6).astype(int)
df['is_Mon'] = (df.index.dayofweek == 0).astype(int)
df['is_Tue'] = (df.index.dayofweek == 1).astype(int)
df['is_Wed'] = (df.index.dayofweek == 2).astype(int)
df['is_Thu'] = (df.index.dayofweek == 3).astype(int)
df['is_Fri'] = (df.index.dayofweek == 4).astype(int)
df.head(3)

In [ ]:
df['is_Jan'] = (df.index.month == 1).astype(int)
df['is_Feb'] = (df.index.month == 2).astype(int)
df['is_Mar'] = (df.index.month == 3).astype(int)
df['is_Apr'] = (df.index.month == 4).astype(int)
df['is_May'] = (df.index.month == 5).astype(int)
df['is_Jun'] = (df.index.month == 6).astype(int)
df['is_Jul'] = (df.index.month == 7).astype(int)
df['is_Aug'] = (df.index.month == 8).astype(int)
df['is_Sep'] = (df.index.month == 9).astype(int)
df['is_Oct'] = (df.index.month == 10).astype(int)
df['is_Nov'] = (df.index.month == 11).astype(int)
df['is_Dec'] = (df.index.month == 12).astype(int)
df.head(3)

In [ ]:
df_train_window = df.tail(187)
df_train_window

In [ ]:
threshold = np.percentile(df_train_window['sales_sum'], 66)
threshold

In [ ]:
df_train_window['is_hotseller'] = np.where(df_train_window['sales_sum'] >= threshold, 1, 0)
df_train_window.head()

In [ ]:
# นับจำนวน is_hot_seller ที่เป็น 1 และ 0
is_hotseller_counts = df_train_window['is_hotseller'].value_counts()

print("จำนวน is_hotseller ที่เป็น 1:", is_hotseller_counts.get(1, 0))
print("จำนวน is_hotseller ที่เป็น 0:", is_hotseller_counts.get(0, 0))

In [ ]:
df_train_window.tail(8)

In [ ]:
df_train_window = df_train_window.reindex(columns=['is_hotseller', 'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11'
                         #, 'year', 'time_index'
                         #, 'isschoolday', 'ismarketday', 'holiday', 'isbuddaday'
                         #, 'is_Sun', 'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri'
                         , 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
])

In [ ]:
test_data = df_train_window.tail(7)
train_data = df_train_window.head(180)

print("Test Data:")
print(test_data)

print("\nTrain Data:")
print(train_data)

In [ ]:
X = train_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11'
                         #, 'year', 'time_index'
                         #, 'isschoolday', 'ismarketday', 'holiday', 'isbuddaday'
                         #, 'is_Sun', 'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri'
                         , 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
]]
y = train_data['is_hotseller']

#Train

มีความจำเป็นต้องกำหนด class_weight='balanced' เนื่องจากปัญหา Imbalance target ซึ่งเกิดจากการต้องการแยกวันสำคัญ ซึ่งคือวันขายดีกว่าปกติ ซึ่งเป็นส่วนน้อย

In [ ]:
# เพิ่ม class_weight='balanced' เข้าไป
#model = LogisticRegression(class_weight='balanced')
model = LogisticRegression()
model.fit(X, y)

In [ ]:
X_test = test_data[['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 'lag_8', 'lag_9', 'lag_10', 'lag_11'
                         #, 'year', 'time_index'
                         #, 'isschoolday', 'ismarketday', 'holiday', 'isbuddaday'
                         #, 'is_Sun', 'is_Mon', 'is_Tue', 'is_Wed', 'is_Thu', 'is_Fri'
                         , 'is_Jan', 'is_Feb', 'is_Mar', 'is_Apr', 'is_May', 'is_Jun', 'is_Jul', 'is_Aug', 'is_Sep', 'is_Oct', 'is_Nov', 'is_Dec'
]]
predictions = model.predict(X_test)

# Print the predictions
print("\nPredictions:")
predictions

#Evaluate

In [ ]:
y_test = test_data['is_hotseller']

In [ ]:
# คำนวณ Confusion Matrix
cm = confusion_matrix(y_test, predictions)

# แสดง Confusion Matrix
print("\nConfusion Matrix:")
print(cm)

In [ ]:
print("\nClassification Report:")
print(classification_report(y_test, predictions))